In [1]:
# 02_feature_engineering.ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/processed/bank_churners_eda_base.csv")

df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  int64  
 1   Attrition_Flag            10127 non-null  object 
 2   Customer_Age              10127 non-null  int64  
 3   Gender                    10127 non-null  object 
 4   Dependent_count           10127 non-null  int64  
 5   Education_Level           10127 non-null  object 
 6   Marital_Status            10127 non-null  object 
 7   Income_Category           10127 non-null  object 
 8   Card_Category             10127 non-null  object 
 9   Months_on_book            10127 non-null  int64  
 10  Total_Relationship_Count  10127 non-null  int64  
 11  Months_Inactive_12_mon    10127 non-null  int64  
 12  Contacts_Count_12_mon     10127 non-null  int64  
 13  Credit_Limit              10127 non-null  float64
 14  Total_

3. Handle Categorical Variables

依據 EDA 的決策：

3.1 Keep Unknown as valid category

不做任何補值
不 drop
不做 Missing Indicator（因為 churn rate 正常）

→ 直接 one-hot encoding 即可

✔ Variables:

Gender

Education_Level

Marital_Status

Income_Category

Card_Category

In [2]:
# Behavioral variables identified from EDA(strongest predictors of churn) 
behavior_vars = [
    "Total_Trans_Ct",
    "Total_Trans_Amt",
    "Months_Inactive_12_mon",
    "Contacts_Count_12_mon",
    "Total_Relationship_Count",
    "Avg_Utilization_Ratio"
]

In [3]:
# Only create bins if they do NOT already exist from EDA

# Tenure Binning
if "Tenure_bin" not in df.columns:
    df["Months_on_book"] = pd.to_numeric(df["Months_on_book"], errors="coerce")
    bins = [0, 24, 36, 48, 60, 100]
    labels = ["<2 years", "2–3 years", "3–4 years", "4–5 years", "5+ years"]
    df["Tenure_bin"] = pd.cut(df["Months_on_book"], bins=bins, labels=labels, right=False)

# Age Binning
if "Age_bin" not in df.columns:
    df["Age_bin"] = pd.cut(
        df["Customer_Age"],
        bins=[18,30,40,50,60,100],
        labels=["18–29","30–39","40–49","50–59","60+"]
    )

# Has_Dependents
if "Has_Dependents" not in df.columns:
    df["Has_Dependents"] = (df["Dependent_count"] > 0).astype(int)


# Has_Dependents
if "Has_Dependents" not in df.columns:
    df["Has_Dependents"] = (df["Dependent_count"] > 0).astype(int)


In [4]:
# Log Transform (from EDA Step E)
log_vars = [
    "Total_Trans_Amt",
    "Total_Revolving_Bal",
    "Avg_Open_To_Buy",
    "Credit_Limit",
    "Total_Amt_Chng_Q4_Q1",
    "Total_Ct_Chng_Q4_Q1"
]

for col in log_vars:
    df[col + "_log"] = np.log1p(df[col])


In [5]:
# One-hot Encode Categorical Variables(keep unknown)
cat_vars = [
    "Gender",
    "Education_Level",
    "Marital_Status",
    "Income_Category",
    "Card_Category",
    "Tenure_bin",
    "Age_bin"
]

df = pd.get_dummies(df, columns=cat_vars, drop_first=True)


In [6]:
# Drop Unused Columns
drop_cols = ["CLIENTNUM", "Attrition_Flag", "Churn_Label"]
df = df.drop(columns=[col for col in drop_cols if col in df.columns])


In [7]:
# Final Dataset: X, y
y = df["Churn_Flag"]
X = df.drop(columns=["Churn_Flag"])

# Save FE Output for modeling
X.to_csv("../data/processed/X_ready.csv", index=False)
y.to_csv("../data/processed/y_ready.csv", index=False)


In [8]:
import pandas as pd

X = pd.read_csv("../data/processed/X_ready.csv")
y = pd.read_csv("../data/processed/y_ready.csv").iloc[:, 0]  # fixed

print("X shape:", X.shape)
print("y shape:", y.shape)

print("\nFirst 5 rows of X:")
display(X.head())

print("\nFirst 10 values of y:")
print(y.head(10))

print("\nUnique values in y (should be 0 and 1):")
print(y.unique())

print("\nAny missing values in X?")
print(X.isna().sum().sum())

print("\nColumns in X (check there is no target or ID):")
print(X.columns.tolist())


X shape: (10127, 46)
y shape: (10127,)

First 5 rows of X:


,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,...,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver,Tenure_bin_3–4 years,Tenure_bin_4–5 years,Tenure_bin_<2 years,Age_bin_30–39,Age_bin_40–49,Age_bin_50–59,Age_bin_60+
0,45,3,39,5,1,3,12691.0,777,11914.0,1.335,...,False,False,False,True,False,False,False,True,False,False
1,49,5,44,6,1,2,8256.0,864,7392.0,1.541,...,False,False,False,True,False,False,False,True,False,False
2,51,3,36,4,1,0,3418.0,0,3418.0,2.594,...,False,False,False,True,False,False,False,False,True,False
3,40,4,34,3,4,1,3313.0,2517,796.0,1.405,...,False,False,False,False,False,False,True,False,False,False
4,40,3,21,5,1,0,4716.0,0,4716.0,2.175,...,False,False,False,False,False,True,True,False,False,False



First 10 values of y:
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: Churn_Flag, dtype: int64

Unique values in y (should be 0 and 1):
[0 1]

Any missing values in X?
0

Columns in X (check there is no target or ID):
['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Has_Dependents', 'Total_Trans_Amt_log', 'Total_Revolving_Bal_log', 'Avg_Open_To_Buy_log', 'Credit_Limit_log', 'Total_Amt_Chng_Q4_Q1_log', 'Total_Ct_Chng_Q4_Q1_log', 'Gender_M', 'Education_Level_Doctorate', 'Education_Level_Graduate', 'Education_Level_High School', 'Education_Level_Post-Graduate', 'Education_Level_Uneducated', 'Education_Level_Unknown', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Income_Category_$40